<a href="https://colab.research.google.com/github/rubuntu/Taller_Introduccion_a_Ciencia_de_Datos_IA_e_Ingenieria_de_Datos/blob/main/sesion_05_regresion_lineal_regresion_logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparativo: Regresión Lineal vs Regresión Logística

# 📘 Regresión Lineal

## 1. Concepto

* Modelo estadístico que **predice un valor numérico continuo** en función de variables independientes.
* Supone una **relación lineal** entre variables:

$$
Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_n X_n + \epsilon
$$

* Donde:

  * $Y$ = variable dependiente (lo que queremos predecir).
  * $X_i$ = variables independientes (predictoras).
  * $\beta_i$ = coeficientes (pendientes).
  * $\epsilon$ = error.

---

## 2. Ejemplo

* Predecir el **precio de una casa** en función de su superficie.

$$
Precio = \beta_0 + \beta_1 \cdot Superficie
$$

Si $\beta_0 = 50,000$ y $\beta_1 = 1,000$,
una casa de 100 m² tendría precio ≈ 150,000.

---

## 3. Gráfico Intuitivo

Una **línea recta** que mejor ajusta los puntos de datos.
La idea es minimizar el error cuadrático medio (**MSE**).

---

## 4. Aplicaciones

* Predicción de ingresos en función de años de educación.
* Forecast de ventas según inversión en marketing.
* Estimar el riesgo crediticio (como variable continua, ej. probabilidad).

---

In [ ]:
# Ejemplo en Python

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X, y = make_regression(n_samples=300, n_features=1, noise=15.0, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

lin_model = LinearRegression().fit(X_train, y_train)
y_pred = lin_model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

plt.scatter(X_test, y_test, alpha=0.5)
order = np.argsort(X_test[:, 0])
plt.plot(X_test[order], y_pred[order], color="red")
plt.title(f"Regresión Lineal: RMSE={rmse:.2f}, R²={r2:.3f}")
plt.xlabel("X"); plt.ylabel("y")
plt.show()


In [ ]:
# --- Gráfico Predicción vs Realidad ---
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()],
         'r--', lw=2)  # línea ideal
plt.xlabel("Valor Real (y_test)")
plt.ylabel("Predicción (y_pred)")
plt.title("Predicción vs Realidad")
plt.show()

# 📘 Regresión Logística

## 1. Concepto

* Variante que sirve para **clasificación binaria** (Sí/No, 0/1).
* En lugar de predecir valores continuos, predice **probabilidades**:

$$
P(Y=1|X) = \frac{1}{1 + e^{-(\beta_0 + \beta_1X_1 + \dots + \beta_nX_n)}}
$$

* La función **sigmoide** transforma cualquier valor en un número entre 0 y 1.

---

## 2. Ejemplo

* Predecir si un cliente **hará churn** (1) o no (0).

$$
Logit(P) = \ln\left(\frac{P}{1-P}\right) = \beta_0 + \beta_1 X
$$

Si el modelo predice $P=0.8$, interpretamos: **80% de probabilidad de churn**.

---

## 3. Gráfico Intuitivo

* La curva de la regresión logística es en forma de **S (sigmoide)**.
* Para valores bajos de X → P cercano a 0.
* Para valores altos de X → P cercano a 1.
* Punto de corte (threshold, usualmente 0.5) → decide clase 0 o 1.

---

## 4. Aplicaciones

* Detección de fraude (fraude = 1, no fraude = 0).
* Diagnóstico médico (enfermo = 1, sano = 0).
* Predicción de abandono (churn).

---

In [ ]:
# Ejemplo en Python

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, RocCurveDisplay

# Dataset binario 1D (válido)
Xc, yc = make_classification(
    n_samples=10000,
    n_features=1,
    n_redundant=0,
    n_informative=1,
    n_classes=2,
    n_clusters_per_class=1,
    class_sep=0.8,      # ↓ separa menos las clases
    flip_y=0.2,         # 20% de etiquetas ruidosas
    random_state=42
)

Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    Xc, yc, test_size=0.25, stratify=yc, random_state=42
)

# Modelo
log_model = LogisticRegression().fit(Xc_train, yc_train)

y_pred = log_model.predict(Xc_test)
y_proba = log_model.predict_proba(Xc_test)[:, 1]

print("Accuracy:", accuracy_score(yc_test, y_pred))
print("F1:", f1_score(yc_test, y_pred))
print("AUC:", roc_auc_score(yc_test, y_proba))

In [ ]:
# Curva sigmoide
xs = np.linspace(Xc_test.min(), Xc_test.max(), 200).reshape(-1, 1)
sig = log_model.predict_proba(xs)[:, 1]

plt.scatter(Xc_test, yc_test, alpha=0.5)
plt.plot(xs, sig)
plt.title("Regresión Logística: Probabilidad P(Y=1|X)")
plt.xlabel("X"); plt.ylabel("Probabilidad")
plt.show()

In [ ]:
# Curva ROC
RocCurveDisplay.from_predictions(yc_test, y_proba)
plt.title("Curva ROC")
plt.show()

In [ ]:
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    PrecisionRecallDisplay, average_precision_score
)

# --- Reporte de clasificación ---
print("\n=== Classification Report ===")
print(classification_report(yc_test, y_pred, digits=3))

# --- Matriz de confusión (texto + plot) ---
cm = confusion_matrix(yc_test, y_pred)
print("\n=== Confusion Matrix ===\n", cm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Matriz de Confusión")
plt.show()

# --- Curva Precision-Recall + Average Precision ---
ap = average_precision_score(yc_test, y_proba)
PrecisionRecallDisplay.from_predictions(yc_test, y_proba)
plt.title(f"Curva Precision-Recall (AP = {ap:.3f})")
plt.show()


# 📊 Diferencias Clave

| Aspecto         | Regresión Lineal      | Regresión Logística        |
| --------------- | --------------------- | -------------------------- |
| Tipo de salida  | Variable continua (ℝ) | Probabilidad (0 a 1)       |
| Función         | Recta                 | Sigmoide (S)               |
| Problema típico | Predicción de valores | Clasificación binaria      |
| Ejemplo         | Precio de casas       | ¿El cliente abandona o no? |
| Métrica común   | MSE, R²               | AUC, F1, LogLoss           |

---

👉 Una buena forma de recordarlo:

* **Lineal = línea recta = predecir números**.
* **Logística = lógica binaria = predecir sí/no con probabilidades**.